In [3]:
# Read in data from read_data.py file
%run "../scripts/read_data.py"

In [ ]:
from pyspark.sql import functions as F

In [115]:
tbl_merchants.limit(5)

name,tags,merchant_abn
Felis Limited,"((furniture, home...",10023283211
Arcu Ac Orci Corp...,"([cable, satellit...",10142254217
Nunc Sed Company,"([jewelry, watch,...",10165489824
Ultricies Digniss...,"([wAtch, clock, a...",10187291046
Enim Condimentum PC,([music shops - m...,10192359162


In [6]:
tbl_merchants.printSchema()

root
 |-- name: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- merchant_abn: long (nullable = true)



In [7]:
tbl_merchants.count()

4026

In [9]:
tbl_merchants.groupBy("tags").count()

tags,count
"[[opticians, opti...",1
([tent and awning...,1
[[motor vehicle s...,1
[[florists suppli...,1
"[[books, periodic...",1
((lawn and garden...,1
((computer progra...,1
"[[cable, satellit...",1
[(motor vehicle s...,1
"[[watch, clock, a...",1


In [22]:
# highest, ... lowest and val as str
# merchants1, 2, 3,... with value 1-5 for the rank

In [28]:
tbl_merchants.first()["tags"]

'((furniture, home furnishings and equipment shops, and manufacturers, except appliances), (e), (take rate: 0.18))'

In [ ]:
# ((, ([, [[, [(

In [116]:
# remove outer brackets in tags
df = tbl_merchants.withColumn("tags", F.regexp_replace("tags", "[\])][\])]", "")) \
        .withColumn("tags", F.regexp_replace("tags", "[\[(][\[(]", "")) 

# separate categories, take rate, and revenue level
# convert take rate to double
df = df.withColumn('categories', F.split(df['tags'], '[)\]], [\[(]').getItem(0)) \
        .withColumn('take_rate', F.split(df['tags'], '[)\]], [\[(]take rate: ').getItem(1).cast("double")) \
        .withColumn('revenue_levels', F.split(df['tags'], '[)\]], [\[(]').getItem(1)) \
        .drop(F.col("tags"))
        

In [117]:
df

name,merchant_abn,categories,take_rate,revenue_levels
Felis Limited,10023283211,"furniture, home f...",0.18,e
Arcu Ac Orci Corp...,10142254217,"cable, satellite,...",4.22,b
Nunc Sed Company,10165489824,"jewelry, watch, c...",4.4,b
Ultricies Digniss...,10187291046,"wAtch, clock, and...",3.29,b
Enim Condimentum PC,10192359162,music shops - mus...,6.33,a
Fusce Company,10206519221,"gift, card, novel...",6.34,a
Aliquam Enim Inco...,10255988167,"computers, comPUt...",4.32,b
Ipsum Primis Ltd,10264435225,"watch, clock, and...",2.39,c
Pede Ultrices Ind...,10279061213,computer programm...,5.71,a
Nunc Inc.,10323485998,"furniture, home f...",6.61,a


In [118]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- merchant_abn: long (nullable = true)
 |-- categories: string (nullable = true)
 |-- take_rate: double (nullable = true)
 |-- revenue_levels: string (nullable = true)

